In [113]:
import pandas as pd
import numpy as np
from simple_factor_process import FactorProcessing_CorrelationBetweenTwoFactors as FactorIC # 因子IC
from simple_factor_process import FactorTool_GetStockSplit as FactorGroup # 分组
from simple_factor_process import FactorPurify # 中性化


import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from statsmodels.api import OLS

以dEP因子为例，测试单因子的绩效

In [114]:
sw_ind = pd.read_pickle('数据/IndexComponent_SWN_I.txt')
pe = pd.read_pickle('数据/StockQuote_PEttm.txt')
stock_close = pd.read_pickle('数据/StockQuote_ClosePrice_BackwardAdj.txt')
stock_open = pd.read_pickle('数据/StockQuote_OpenPrice_BackwardAdj.txt')

In [173]:
famafrench = pd.read_excel("STK_MKT_THRFACDAY.xlsx")
famafrench.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,MarkettypeID,TradingDate,RiskPremium1,RiskPremium2,SMB1,SMB2,HML1,HML2
0,股票市场类型编码,交易日期,市场风险溢价因子(流通市值加权),市场风险溢价因子(总市值加权),市值因子(流通市值加权),市值因子(总市值加权),账面市值比因子(流通市值加权),账面市值比因子(总市值加权)
1,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位,没有单位
2,P9706,1990-12-19,2.473374,2.309671,NaN,NaN,NaN,NaN
3,P9710,1990-12-19,2.473374,2.309671,NaN,NaN,NaN,NaN
4,P9709,1990-12-19,2.473374,2.309671,NaN,NaN,NaN,NaN


In [174]:
# 9706：沪深A股市场（不包含科创板、创业板）
# RiskPremium1 [市场风险溢价因子(流通市值加权)] - 考虑现金红利再投资的日市场回报率(流通市值加权平均法)与日度化无风险利率之差（央行公布三月定存基准利率）。
# RiskPremium2 [市场风险溢价因子(总市值加权)] - 考虑现金红利再投资的日市场回报率(总市值加权平均法)与日度化无风险利率之差（央行公布三月定存基准利率）。
# SMB1 [市值因子(流通市值加权)] - 小盘股组合和大盘股组合的收益率之差，组合划分基于FAMA 2*3组合划分方法。组合日收益率的计算采用流通市值加权计算。
# SMB2 [市值因子(总市值加权)] - 小盘股组合和大盘股组合的收益率之差，组合划分基于FAMA 2*3组合划分方法。组合日收益率的计算采用总市值加权计算。
# HML1 [账面市值比因子(流通市值加权)] - 高账面市值比组合和低账面市值比组合的收益率之差，组合划分基于FAMA 2*3组合划分方法。组合投资收益率的计算采用流通市值加权。
# HML2 [账面市值比因子(总市值加权)] - 高账面市值比组合和低账面市值比组合的收益率之差，组合划分基于FAMA 2*3组合划分方法。组合投资收益率的计算采用总市值加权。

famafrench = famafrench[famafrench['MarkettypeID'] == "P9706"]
famafrench = famafrench.reset_index(drop= True)
famafrench = famafrench.set_index("TradingDate").loc["2012-01-01" : '2023-12-31'][['RiskPremium2', "SMB2", "HML2"]]
famafrench

,RiskPremium2,SMB2,HML2
TradingDate,,,
2012-01-04,-0.016205,-0.004502,0.00939
2012-01-05,-0.013403,-0.027523,0.01463
2012-01-06,0.00707,-0.002101,0.000992
2012-01-09,0.030277,0.008748,-0.010063
2012-01-10,0.028804,0.006326,-0.014061
...,...,...,...
2023-12-25,0.00202,-0.00887,-0.001847
2023-12-26,-0.005552,-0.005357,0.004727
2023-12-27,0.0064,0.002296,0.002342


In [126]:
# 构建方法：涨跌停的比列
ZT = (stock_close - stock_open) / stock_open 
number_of_stocks = ZT.T.count()
ZT = abs(ZT) >= 0.09 
number_of_stocks_toomuch = ZT.T.sum()
factor = number_of_stocks_toomuch / number_of_stocks
factor.replace(np.nan, 0, inplace = True) 
# select 2012 - 2023
factor = factor.loc['20120101':'20231231']
factor

20120104    0.011760
20120105    0.027439
20120106    0.005220
20120109    0.028273
20120110    0.026945
              ...   
20231225    0.013508
20231226    0.010692
20231227    0.008815
20231228    0.018748
20231229    0.015370
Length: 2915, dtype: float64

In [171]:
daily_returns = stock_close.pct_change()
daily_returns = daily_returns.loc['20120101':'20231231', :]
daily_returns

/var/folders/md/wt8g61857ld_20f_r6rn4nk40000gn/T/ipykernel_3227/1175375890.py:1: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  daily_returns = stock_close.pct_change()


,000001.SZ,000002.SZ,000003.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,873570.BJ,873576.BJ,873593.BJ,873665.BJ,873679.BJ,873693.BJ,873703.BJ,873726.BJ,873833.BJ,T00018.SH
20120104,-0.027582,-0.018742,NaN,-0.022250,0.000000,-0.009756,0.050152,-0.050215,-0.073394,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
20120105,0.015172,-0.010914,NaN,-0.073325,0.000000,-0.012315,-0.050651,-0.040785,-0.055446,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
20120106,-0.002599,-0.012414,NaN,0.004093,0.000000,-0.002494,-0.028963,0.006299,0.059748,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
20120109,0.028013,0.034916,NaN,0.046196,0.000000,0.027500,0.039246,0.050078,0.099901,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
20120110,0.000000,0.016194,NaN,0.059740,0.000000,0.036496,0.024169,0.050671,0.042266,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20231225,-0.001087,-0.007707,NaN,-0.017511,-0.045872,-0.011161,-0.010309,-0.008511,-0.003433,-0.009677,...,0.048374,0.078667,0.018418,0.062807,0.032244,0.019663,0.045506,0.171841,0.038771,0.0
20231226,-0.009793,-0.012621,NaN,-0.024825,0.009615,-0.015801,-0.008333,-0.004292,-0.006891,-0.013029,...,0.043079,-0.012441,-0.029547,0.020314,0.000434,-0.047291,0.024483,0.036201,-0.009155,0.0
20231227,0.002198,0.000000,NaN,0.022846,0.000000,0.043578,0.004202,0.008621,0.008673,0.026403,...,0.045521,0.042137,-0.033333,-0.021719,-0.006938,0.020241,0.027616,0.033671,0.018479,0.0
20231228,0.036184,0.034415,NaN,0.007020,0.038095,0.000000,0.018828,0.004274,0.012038,0.003215,...,0.004213,0.012922,0.004073,0.071230,0.006987,-0.019839,-0.000517,-0.035268,0.034194,0.0


In [172]:
# DO NOT RUN IT TWICE
factor.index = pd.to_datetime(factor.index)

daily_returns.index = pd.to_datetime(daily_returns.index)

monthly_trading_day = pd.read_pickle('数据/monthly_trading_day.pkl')
monthly_trading_day['start_date'] = pd.to_datetime(monthly_trading_day['start_date'], format='%Y%m%d')
monthly_trading_day['end_date'] = pd.to_datetime(monthly_trading_day['end_date'], format='%Y%m%d')

start_date = pd.to_datetime('20120101', format='%Y%m%d')
end_date = pd.to_datetime('20231231', format='%Y%m%d')

filtered_trading_days = monthly_trading_day.loc[(monthly_trading_day['end_date'] >= start_date) & (monthly_trading_day['end_date'] <= end_date)]
filtered_trading_days

,start_date,end_date
24,2012-01-04,2012-01-31
25,2012-02-01,2012-02-29
26,2012-03-01,2012-03-30
27,2012-04-05,2012-04-27
28,2012-05-02,2012-05-31
...,...,...
162,2023-07-03,2023-07-31
163,2023-08-01,2023-08-31
164,2023-09-01,2023-09-28
165,2023-10-09,2023-10-31


In [138]:
from statsmodels.regression.rolling import RollingOLS

window_size = 20

for stock in daily_returns.columns:
    # 我们先不弄走nan
    Y = daily_returns[stock]
    Y.index = pd.to_datetime(Y.index)
    X = factor.loc[Y.index]
    results = {}  # Dictionary to store regression results
    # print(y, X)
    # break

    for index, row in monthly_trading_day.iterrows():
        start_date = pd.to_datetime(row['start_date'], format='%Y%m%d')
        end_date = pd.to_datetime(row['end_date'], format='%Y%m%d')

                # Locate the subset of X and Y data for this month
        X_subset = X.loc[(X.index >= start_date) & (X.index <= end_date)]
        Y_subset = Y.loc[(Y.index >= start_date) & (Y.index <= end_date)]

                # Check if there's enough data to perform the regression
        if not X_subset.empty and not Y_subset.empty and len(X_subset) == len(Y_subset):
            # Add a constant to the independent variables for the intercept
            X_with_const = sm.add_constant(X_subset)
            
            # Run the regression
            model = sm.OLS(Y_subset, X_with_const)
            regression_results = model.fit()
            
            # Store the regression results
            results[end_date] = regression_results

            print(regression_results.params)

            break
    break





    
    # # Loop over the end dates of each month
    # for index, row in filtered_trading_days.iterrows():
    #     end_date = row['end_date']  # This is already a datetime object, no conversion needed

    #     # Use 'get_loc' with 'pad' to find the index of the last available date in 'y' up to 'end_date'
    #     end_loc = y.index.get_loc(end_date, method='pad')
        
    #     # Select the data up to the end of the month using the located index
    #     y_month = y.iloc[end_loc + 1 - window_size:end_loc + 1]
    #     X_month = X.iloc[end_loc + 1 - window_size:end_loc + 1]

    #     # Check if there's enough data to perform the rolling regression
    #     if len(y_month) == window_size and len(X_month) == window_size:
    #         rolling_model = RollingOLS(y_month, X_month, window=window_size)
    #         # Fit the model
    #         rolling_betas_month = rolling_model.fit()
    #         # Store the coefficients/betas for this month-end
    #         rolling_betas = rolling_betas.append(rolling_betas_month.params.tail(1))

    # # Remove this break to process all stocks
    # break


const    0.004785
0       -0.014943
dtype: float64


In [198]:
factor

2012-01-04    0.011760
2012-01-05    0.027439
2012-01-06    0.005220
2012-01-09    0.028273
2012-01-10    0.026945
                ...   
2023-12-25    0.013508
2023-12-26    0.010692
2023-12-27    0.008815
2023-12-28    0.018748
2023-12-29    0.015370
Length: 2915, dtype: float64

In [230]:
fourfactors = pd.merge(pd.DataFrame(factor).reset_index(), famafrench.reset_index() , left_index=True, right_index=True, how='left')
fourfactors.columns = ["Tradingdays","APL", "TradingDate", "RiskPremium2", "SMB2", "HML2"]
fourfactors_reordered = fourfactors[[ 'TradingDate','APL', 'RiskPremium2', 'SMB2', 'HML2']]
fourfactors_reordered = fourfactors_reordered.set_index('TradingDate')
fourfactors_reordered

,APL,RiskPremium2,SMB2,HML2
TradingDate,,,,
2012-01-04,0.011760,-0.016205,-0.004502,0.00939
2012-01-05,0.027439,-0.013403,-0.027523,0.01463
2012-01-06,0.005220,0.00707,-0.002101,0.000992
2012-01-09,0.028273,0.030277,0.008748,-0.010063
2012-01-10,0.026945,0.028804,0.006326,-0.014061
...,...,...,...,...
2023-12-25,0.013508,0.00202,-0.00887,-0.001847
2023-12-26,0.010692,-0.005552,-0.005357,0.004727
2023-12-27,0.008815,0.0064,0.002296,0.002342


In [ ]:
for stock in daily_returns.columns:
    Y = daily_returns[stock]
    Y.index = pd.to_datetime(Y.index)


In [ ]:


# Make sure that both 'factor' and 'daily_returns' indexes are datetime
factor.index = pd.to_datetime(factor.index)
daily_returns.index = pd.to_datetime(daily_returns.index)

# Initialize the results dictionary
all_results = {}

# Loop through each stock
for stock in daily_returns.columns:
    
    Y = daily_returns[stock]
    
    # Make sure both Y and X are aligned by using the intersection of their indexes
    common_index = Y.index.intersection(factor.index)
    Y = Y[common_index]
    X = factor.loc[common_index]
    
    # Dictionary to store regression results for the current stock
    stock_results = {}

    for index, row in monthly_trading_day.iterrows():
        # Convert the start and end dates to datetime
        start_date = row['start_date']
        end_date = row['end_date']

        # Locate the subset of X and Y data for this month
        X_subset = X.loc[(X.index >= start_date) & (X.index <= end_date)]
        Y_subset = Y.loc[(Y.index >= start_date) & (Y.index <= end_date)]

        # Check if there's enough data to perform the regression
        if not X_subset.empty and not Y_subset.empty and len(X_subset) == len(Y_subset):
            # Add a constant to the independent variables for the intercept
            X_with_const = sm.add_constant(X_subset, has_constant='add')
            
            # Run the regression
            model = sm.OLS(Y_subset, X_with_const)
            regression_results = model.fit()
            
            # Store the regression results for this end_date
            stock_results[end_date] = regression_results.params

    # Store the results for the current stock
    all_results[stock] = stock_results

# Convert all_results to a DataFrame for a clean and organized output
all_params = {(outerKey, innerKey): values for outerKey, innerDict in all_results.items() for innerKey, values in innerDict.items()}
param_df = pd.DataFrame(all_params).T
param_df.index.names = ['Stock', 'End_Date']
print(param_df)


                         const         0
Stock     End_Date                      
000001.SZ 2012-01-31  0.004785 -0.014943
          2012-02-29 -0.005669  1.312438
          2012-03-30 -0.000052 -0.612668
          2012-04-27 -0.002393  0.637096
          2012-05-31 -0.008816  1.392773
...                        ...       ...
T00018.SH 2023-07-31  0.000000  0.000000
          2023-08-31  0.000000  0.000000
          2023-09-28  0.000000  0.000000
          2023-10-31  0.000000  0.000000
          2023-11-30  0.000000  0.000000

[797082 rows x 2 columns]


In [227]:
daily_returns

,000001.SZ,000002.SZ,000003.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,...,873570.BJ,873576.BJ,873593.BJ,873665.BJ,873679.BJ,873693.BJ,873703.BJ,873726.BJ,873833.BJ,T00018.SH
2012-01-04,-0.027582,-0.018742,NaN,-0.022250,0.000000,-0.009756,0.050152,-0.050215,-0.073394,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2012-01-05,0.015172,-0.010914,NaN,-0.073325,0.000000,-0.012315,-0.050651,-0.040785,-0.055446,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2012-01-06,-0.002599,-0.012414,NaN,0.004093,0.000000,-0.002494,-0.028963,0.006299,0.059748,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2012-01-09,0.028013,0.034916,NaN,0.046196,0.000000,0.027500,0.039246,0.050078,0.099901,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2012-01-10,0.000000,0.016194,NaN,0.059740,0.000000,0.036496,0.024169,0.050671,0.042266,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-25,-0.001087,-0.007707,NaN,-0.017511,-0.045872,-0.011161,-0.010309,-0.008511,-0.003433,-0.009677,...,0.048374,0.078667,0.018418,0.062807,0.032244,0.019663,0.045506,0.171841,0.038771,0.0
2023-12-26,-0.009793,-0.012621,NaN,-0.024825,0.009615,-0.015801,-0.008333,-0.004292,-0.006891,-0.013029,...,0.043079,-0.012441,-0.029547,0.020314,0.000434,-0.047291,0.024483,0.036201,-0.009155,0.0
2023-12-27,0.002198,0.000000,NaN,0.022846,0.000000,0.043578,0.004202,0.008621,0.008673,0.026403,...,0.045521,0.042137,-0.033333,-0.021719,-0.006938,0.020241,0.027616,0.033671,0.018479,0.0
2023-12-28,0.036184,0.034415,NaN,0.007020,0.038095,0.000000,0.018828,0.004274,0.012038,0.003215,...,0.004213,0.012922,0.004073,0.071230,0.006987,-0.019839,-0.000517,-0.035268,0.034194,0.0


In [139]:
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm
import pandas as pd

# Make sure that both 'factor' and 'daily_returns' indexes are datetime
factor.index = pd.to_datetime(factor.index)
daily_returns.index = pd.to_datetime(daily_returns.index)

# Initialize the results dictionary
all_results = {}

# Loop through each stock
for stock in daily_returns.columns:
    
    Y = daily_returns[stock]
    
    # Make sure both Y and X are aligned by using the intersection of their indexes
    common_index = Y.index.intersection(factor.index)
    Y = Y[common_index]
    X = factor.loc[common_index]
    
    # Dictionary to store regression results for the current stock
    stock_results = {}

    for index, row in monthly_trading_day.iterrows():
        # Convert the start and end dates to datetime
        start_date = row['start_date']
        end_date = row['end_date']

        # Locate the subset of X and Y data for this month
        X_subset = X.loc[(X.index >= start_date) & (X.index <= end_date)]
        Y_subset = Y.loc[(Y.index >= start_date) & (Y.index <= end_date)]

        # Check if there's enough data to perform the regression
        if not X_subset.empty and not Y_subset.empty and len(X_subset) == len(Y_subset):
            # Add a constant to the independent variables for the intercept
            X_with_const = sm.add_constant(X_subset, has_constant='add')
            
            # Run the regression
            model = sm.OLS(Y_subset, X_with_const)
            regression_results = model.fit()
            
            # Store the regression results for this end_date
            stock_results[end_date] = regression_results.params

    # Store the results for the current stock
    all_results[stock] = stock_results

# Convert all_results to a DataFrame for a clean and organized output
all_params = {(outerKey, innerKey): values for outerKey, innerDict in all_results.items() for innerKey, values in innerDict.items()}
param_df = pd.DataFrame(all_params).T
param_df.index.names = ['Stock', 'End_Date']
print(param_df)


                         const         0
Stock     End_Date                      
000001.SZ 2012-01-31  0.004785 -0.014943
          2012-02-29 -0.005669  1.312438
          2012-03-30 -0.000052 -0.612668
          2012-04-27 -0.002393  0.637096
          2012-05-31 -0.008816  1.392773
...                        ...       ...
T00018.SH 2023-07-31  0.000000  0.000000
          2023-08-31  0.000000  0.000000
          2023-09-28  0.000000  0.000000
          2023-10-31  0.000000  0.000000
          2023-11-30  0.000000  0.000000

[797082 rows x 2 columns]


In [ ]:
ep = 1/pe
ep.replace(np.inf, np.nan, inplace = True) #  计算EPttm并去除无限大的值
ep.replace(-np.inf, np.nan, inplace = True) #  计算EPttm并去除无限大的值
dEP = ep.diff(60) # dEP因子是EP因子的六十日delta
dEP_monthly = dEP.reindex(index = monthly_trading_day.end_date) # 按月抽样得到dEP的因子值
stock_ret_monthly = stock_close.reindex(index = monthly_trading_day.end_date).pct_change() #计算股票每月收益
stock_ret_monthly_nextopen = stock_open.shift(-1).reindex(monthly_trading_day.end_date).pct_change()

/var/folders/md/wt8g61857ld_20f_r6rn4nk40000gn/T/ipykernel_16750/1515960077.py:6: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  stock_ret_monthly = stock_close.reindex(index = monthly_trading_day.end_date).pct_change() #计算股票每月收益
/var/folders/md/wt8g61857ld_20f_r6rn4nk40000gn/T/ipykernel_16750/1515960077.py:7: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  stock_ret_monthly_nextopen = stock_open.shift(-1).reindex(monthly_trading_day.end_date).pct_change()


In [ ]:
dEP_monthly

In [ ]:
def simple_factor_test(factor,use_data = 'this_close'):
    if use_data=='this_close':
        this_ret_data = stock_ret_monthly.shift(-1)
    else: 
        this_ret_data = stock_ret_monthly_nextopen.shift(-1)
    ic,rankic = FactorIC(factor,this_ret_data) # 计算因子的ic,rankic序列
    factor_group = FactorGroup(factor)
    condata = pd.concat([factor_group.unstack(),this_ret_data.unstack()],axis=1).dropna().reset_index()
    condata.columns =['stockcode','date','group_id','ret']
    group_ret = condata.groupby(['date','group_id'])['ret'].mean().unstack()
    return ic,rankic,group_ret

In [ ]:
ic,rankic,group_ret  = simple_factor_test(dEP_monthly)

In [ ]:
ic

end_date
20100129   -0.016133
20100226    0.082856
20100331   -0.041014
20100430   -0.100746
20100531    0.056532
              ...   
20230731   -0.011495
20230831   -0.010935
20230928   -0.070958
20231031   -0.014430
20231130         NaN
Length: 167, dtype: float64

In [ ]:
group_ret

group_id,0.0,1.0,2.0,3.0,4.0
date,,,,,
20100129,0.056457,0.067620,0.073110,0.073637,0.050249
20100226,0.010306,0.032678,0.044475,0.054703,0.054152
20100331,-0.073086,-0.065382,-0.077752,-0.051820,-0.081418
20100430,-0.068712,-0.058613,-0.060347,-0.085509,-0.105574
20100531,-0.099717,-0.093635,-0.086971,-0.085237,-0.076485
...,...,...,...,...,...
20230630,0.000839,-0.012075,-0.006102,0.002027,0.031755
20230731,-0.044249,-0.036021,-0.038696,-0.041185,-0.051512
20230831,-0.003965,-0.003497,-0.006606,0.000582,0.001199
